## PART 1

In [1]:
import pandas as pd

df = pd.read_csv('Lab2_dataset.csv')
print(df.head())

   Unnamed: 0 label                                               text   
0         605   ham  Subject: enron methanol ; meter # : 988291\r\n...  \
1        2349   ham  Subject: hpl nom for january 9 , 2001\r\n( see...   
2        3624   ham  Subject: neon retreat\r\nho ho ho , we ' re ar...   
3        4685  spam  Subject: photoshop , windows , office . cheap ...   
4        2030   ham  Subject: re : indian springs\r\nthis deal is t...   

   label_num  
0          0  
1          0  
2          0  
3          1  
4          0  


In [2]:
X = df['text']
y = df['label']

from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer(max_features=1000)  
X_vectorized = vectorizer.fit_transform(X)
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_vectorized, y, test_size=0.2, random_state=42)


## PART A

Model Training and Evaluation

In [3]:
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB, MultinomialNB
from sklearn.metrics import accuracy_score, classification_report

svc_classifier = SVC()
svc_classifier.fit(X_train, y_train)
svc_predictions = svc_classifier.predict(X_test)
svc_accuracy = accuracy_score(y_test, svc_predictions)

gnb_classifier = GaussianNB()
gnb_classifier.fit(X_train.toarray(), y_train)
gnb_predictions = gnb_classifier.predict(X_test.toarray())
gnb_accuracy = accuracy_score(y_test, gnb_predictions)

mnb_classifier = MultinomialNB()
mnb_classifier.fit(X_train, y_train)
mnb_predictions = mnb_classifier.predict(X_test)
mnb_accuracy = accuracy_score(y_test, mnb_predictions)



In [4]:
print("Support Vector Classifier (SVC) Accuracy:", svc_accuracy)
print("Gaussian Naive Bayes Accuracy:", gnb_accuracy)
print("Multinomial Naive Bayes Accuracy:", mnb_accuracy)

print("\nClassification Report for SVC:")
print(classification_report(y_test, svc_predictions))

print("\nClassification Report for Gaussian Naive Bayes:")
print(classification_report(y_test, gnb_predictions))

print("\nClassification Report for Multinomial Naive Bayes:")
print(classification_report(y_test, mnb_predictions))


Support Vector Classifier (SVC) Accuracy: 0.9497584541062802
Gaussian Naive Bayes Accuracy: 0.9420289855072463
Multinomial Naive Bayes Accuracy: 0.9342995169082126

Classification Report for SVC:
              precision    recall  f1-score   support

         ham       0.98      0.95      0.96       742
        spam       0.88      0.96      0.92       293

    accuracy                           0.95      1035
   macro avg       0.93      0.95      0.94      1035
weighted avg       0.95      0.95      0.95      1035


Classification Report for Gaussian Naive Bayes:
              precision    recall  f1-score   support

         ham       0.99      0.93      0.96       742
        spam       0.85      0.97      0.90       293

    accuracy                           0.94      1035
   macro avg       0.92      0.95      0.93      1035
weighted avg       0.95      0.94      0.94      1035


Classification Report for Multinomial Naive Bayes:
              precision    recall  f1-score   sup

## PART B

## These statistics give a sense of the overall distribution of prices in the dataset and can help us to identify potential outliers or anomalies

In [5]:
df2 = pd.read_csv('AB_NYC_2019.csv')
df2.head()
print(df2['price'].describe())

count    48895.000000
mean       152.720687
std        240.154170
min          0.000000
25%         69.000000
50%        106.000000
75%        175.000000
max      10000.000000
Name: price, dtype: float64


In [6]:
from scipy import stats

z_scores = stats.zscore(df2['price'])
threshold = 3

df2_no_outliers_z = df2[(z_scores < threshold)]

print("Number of outliers removed using Z-score approach:", len(df2) - len(df2_no_outliers_z))
print(df2_no_outliers_z['price'].describe())


Number of outliers removed using Z-score approach: 388
count    48507.000000
mean       138.746903
std        107.558233
min          0.000000
25%         69.000000
50%        105.000000
75%        175.000000
max        860.000000
Name: price, dtype: float64


In [7]:
Q1 = df2['price'].quantile(0.25)
Q3 = df2['price'].quantile(0.75)

IQR = Q3 - Q1

lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

df2_no_outliers_whiskers = df2[(df2['price'] >= lower_bound) & (df2['price'] <= upper_bound)]

print("Number of outliers removed using whiskers (IQR) approach:", len(df2) - len(df2_no_outliers_whiskers))
print("\nStatistics after whiskers approach:")
print(df2_no_outliers_whiskers['price'].describe())


Number of outliers removed using whiskers (IQR) approach: 2972

Statistics after whiskers approach:
count    45923.000000
mean       119.970320
std         68.150148
min          0.000000
25%         65.000000
50%        100.000000
75%        159.000000
max        334.000000
Name: price, dtype: float64


In [8]:

df2_no_outliers_z.to_csv('Cleaned_Data_Z_Score.csv', index=False)
df2_no_outliers_whiskers.to_csv('Cleaned_Data_Whiskers.csv', index=False)


From the results : -

- Removing outliers reduced the mean price because outliers can strongly influence and skew the mean towards extreme values.
- Removing outliers decreased the standard deviation, indicating that the data is now more tightly clustered around the mean.
- Outliers removal made the highest values smaller by getting rid of extreme high values in the data.
- The percentiles' values have remained relatively consistent, that means the overall average of the data has not undergone significant alterations.